In [1]:
from datetime import datetime, timezone, timedelta
import pandas as pd

from database.market import Market
from database.sec import SEC
from processor.processor import Processor as p
from modeler_strats.universal_modeler import UniversalModeler
from fund.fund import Fund

from pricer.pricer import Pricer as pricer_list
from ranker.ranker import Ranker as ranker_list
from classifier.classifier import Classifier as classifier_list
from portfolio.aportfolio import APortfolio

ModuleNotFoundError: No module named 'database'

## If you're running this Q4 slice the filing set to only include Q3

In [17]:
lookupidiot

NameError: name 'lookupidiot' is not defined

In [2]:
training_year = 4
start_year = datetime.now().year
end_year = start_year + 1
strat_class = Financial(training_year)
strat_class.pull_included_columns()
strat_class.pull_factors()

In [3]:
market = Market()
sec = SEC()
market.connect()
sp5 = market.retrieve("sp500")
deactivated_tickers = market.retrieve("deactivated_tickers")
unmodeled_tickers = market.retrieve("unmodeled_tickers")
market.disconnect()

In [4]:
tickers = [x for x in sp5["Symbol"] if x not in list(deactivated_tickers["ticker"]) and x not in list(unmodeled_tickers["tickers"])]

In [16]:
sec.connect()
market.connect()
strat_class.db.connect()
filings_list = []
for ticker in tqdm(tickers):
    try:
        row = sp5[sp5["Symbol"]==ticker]
        cik = int(row["CIK"])
        filing_data = sec.retrieve_filing_data(cik)
        filing_data["date"] = [datetime.strptime(str(x),"%Y%m%d") for x in filing_data["filed"]]
        filing_data = p.column_date_processing(filing_data)
        filing_data["ticker"] = ticker
        prices = market.retrieve_ticker_prices("prices",ticker)
        prices = p.column_date_processing(prices)
        prices.sort_values("date",inplace=True)
        complete = strat_class.transform_live(prices,filing_data)
        filings_list.append(complete[complete["year"]==start_year].reset_index(drop=True))
    except Exception as e:
        print(ticker,str(e))
market.disconnect()
strat_class.db.connect()
sec.disconnect()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 405/405 [02:13<00:00,  3.04it/s]


In [17]:
filings = pd.concat(filings_list)

In [18]:
filings = filings.groupby(["year","quarter","ticker"]).mean().reset_index()

In [19]:
filings

,year,quarter,ticker,adjclose,assets,liabilitiesandstockholdersequity,earningspersharediluted
0,2021,1,A,122.519562,9.650500e+09,9.650500e+09,0.780000
1,2021,1,AAL,19.287377,6.100150e+10,6.100150e+10,-1.104286
2,2021,1,AAP,162.537750,1.154408e+10,1.154408e+10,3.063636
3,2021,1,AAPL,127.162256,3.389710e+11,3.389710e+11,1.465000
4,2021,1,ABC,105.817997,4.506080e+10,4.506080e+10,1.355000
...,...,...,...,...,...,...,...
1603,2021,4,XYL,122.686414,8.450500e+09,8.450500e+09,0.785000
1604,2021,4,YUM,125.769045,6.135500e+09,6.135500e+09,2.157500
1605,2021,4,ZBH,130.113106,2.421315e+10,2.421315e+10,0.467500
1606,2021,4,ZBRA,567.755312,5.620000e+09,5.620000e+09,5.870000


In [22]:
strat_class.db.connect()
models = strat_class.db.retrieve("models")
models["model"] = [pickle.loads(x) for x in models["model"]]
for year in tqdm(range(start_year,int(end_year))):
    for quarter in tqdm(range(4,5)):
        try:
            prediction_set = strat_class.prediction_set(filings,year)
            prediction_set = prediction_set[prediction_set["quarter"]==quarter]
            prediction_set = m.predict(models,prediction_set,strat_class.factors)
            prediction_set = strat_class.prediction_clean(prediction_set)
            strat_class.db.store("recs",prediction_set)
        except Exception as e:
            print(str(e))
            continue
strat_class.db.disconnect()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 17.24it/s]


In [21]:
prediction_set

,skl_prediction,skl_score,xgb_prediction,xgb_score,cat_prediction,cat_score,year,quarter,ticker,training_year
1206,160.336720,0.983629,160.336945,0.984969,163.041268,0.985642,2022,1,A,4
1207,22.988414,0.983629,23.005276,0.984969,21.421279,0.985642,2022,1,AAL,4
1208,231.151818,0.983629,231.151718,0.984969,222.374780,0.985642,2022,1,AAP,4
1209,162.152434,0.983629,162.254608,0.984969,167.720691,0.985642,2022,1,AAPL,4
1210,127.728407,0.983629,127.740051,0.984969,132.585686,0.985642,2022,1,ABC,4
...,...,...,...,...,...,...,...,...,...,...
1603,127.801726,0.983629,127.801147,0.984969,129.722072,0.985642,2022,1,XYL,4
1604,130.437460,0.983629,130.436630,0.984969,127.708213,0.985642,2022,1,YUM,4
1605,135.613721,0.983629,135.617783,0.984969,136.102344,0.985642,2022,1,ZBH,4
1606,586.767578,0.983629,586.759033,0.984969,579.150839,0.985642,2022,1,ZBRA,4
